# Background

This RNA-seq workflow is adapted from the RNA-seq tutuorial at [rnabio.org](https://rnabio.org).

``` Malachi Griffith*, Jason R. Walker, Nicholas C. Spies, Benjamin J. Ainscough, Obi L. Griffith*. 2015. Informatics for RNA-seq: A web resource for analysis on the cloud. PLoS Comp Biol. 11(8):e1004393```

The [ENCODE project](https://www.encodeproject.org) publishes [guidelines and best practices for RNA-seq](https://www.encodeproject.org/documents/cede0cbe-d324-4ce7-ace4-f0c3eddf5972/@@download/attachment/ENCODE%20Best%20Practices%20for%20RNA_v2.pdf) that are a must read.

## Initial setup
The tutorial being followed uses AWS cloud but I am writing this pipeline to be ran locally on a linux-based machine. More specifically, I am using Manjaro with kernel version 6.1.7-1-MANJARO (64-bit) and KDE plasma version 5.26.4 on X11.

[Configure your environment](https://rnabio.org/module-00-setup/0000/09/01/Environment/) and then [install all necessary software](https://rnabio.org/module-00-setup/0000/10/01/Installation/) according to the rnabio website. The specific steps you follow while doing this will depend on your system and how you want to install the tools. I would suggest looking for maintained packages for your specific distribution before trying to install from source.

Specific verison information of software installed on my setup:
- SAMtools v1.16.1, installed from AUR
    - run with ```samtools``` from CL
- bam-readcount v1.0.1, compiled from source
    - run from ```~$HOME/bioinformatics/bam-readcount/build/bin``` with ```./bam-readcount```
- HISAT2 v2.2.1-1, installed from AUR
    - run with ```hisat2``` from CL
- stringtie v2.2.1, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/stringtie-2.2.1.Linux_x86_64``` with ```./stringtie```
- gffcompare v0.12.6-2, installed from AUR
    - run with ```gffcompare``` from CL
- htseq-count v.0.11.3-1, installed from AUR and also in bioinformatics conda env
    - run with ```htseq-count``` from terminal with conda bioinformatics env active
- tophat v2.1.1, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/tophat-2.1.1.Linux_x86_64``` with ```./gtf_to_fasta```
- kallisto v0.48.0-1, installed from AUR
    - run with ```kallisto``` from CL
- fastqc v0.11.9-1, installed from AUR
    - run with ```fastqc``` from CL
- multiqc v1.14, installed with pip3 in bioinformatics conda env
    - run with ```multiqc --help``` from CL
- picard v2.26.4, downloaded as JAR file
    - run from ```~$HOME/bioinformatics/``` with ```java -jar picard.jar```
- flexbar v3.5.0, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/flexbar-3.5.0-linux``` with ```./flexbar```
- regtools v1.0.0, installed from AUR (git source)
    - run with ```regtools``` from CL
- rseqc v5.0.1, installed with pip3 in bioinformatics conda env
    - import in python with ```rseqc``` or run with ```read_GC.py``` in CL?
- bedops v2.4.40, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/bedops_linux_x86_64-v2.4.40/bin``` with```./bedops``` and ```./gff2bed```
- gtfToGenePred v unknown, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/gtfToGenePred``` with ```./gtfToGenePred```
- genePredToBed v unknown, downloaded pre-compiled binary
    - run from ```~$HOME/bioinformatics/genePredToBed``` with ```./genePredToBed```
- how_are_we_stranded_here, v unknown, installed with pip3 in bioinformatics conda env
    - run with ```check_strandedness``` in CL
- cell ranger v7.1.0, have to register to download, pre-compiled binary
    - run from ```~$HOME/bioinformatics/cellranger-7.1.0``` with ```./cellranger```
- tabix, website only offers apt installation, not relevant on arch-based distro. Tabix seems to be included with HTSlib which is already installed as a dependency for samtools and bcftools
- BWA v0.7.17=r1188, compiled from source
    - run from ```~$HOME/bioinformatics/bwa``` with ```./bwa``` or ```bwa```
- BCFtools v1.16, installed from AUR
    - run with ```bcftools``` from CL
- peddy v0.4.8, cloned and installed with pip in bioinformatics conda env
    - run with ```peddy``` from CL in conda env
- slivar v0.2.7, pre-complied binary
    - run from ```~$HOME/bioinformatics``` with ```./slivar```
- STRling v0.5.1, pre-complied binary
    - run from ```~$HOME/bioinformatics``` with ```./strling```
- freebayes v1.3.6, pre-compiled binary
    - run from ```~$HOME/bioinformatics``` with ```./freebayes-1.3.6-linux-amd64-static```

### R Libraries needed:
- devtools
- dplyr
- gplots
- ggplot2
- UpSetR

install with 
```install.packages(c("devtools","dplyr","gplots","ggplot2","UpSetR"),repos="http://cran.us.r-project.org")```

### Bioconductor libraries needed:
- [genefilter](http://bioconductor.org/packages/release/bioc/html/genefilter.html)
- [ballgown](http://bioconductor.org/packages/release/bioc/html/ballgown.html)
- [edgeR](http://www.bioconductor.org/packages/release/bioc/html/edgeR.html)
- [GenomicRanges](http://bioconductor.org/packages/release/bioc/html/GenomicRanges.html)
- [rhdf5](https://www.bioconductor.org/packages/release/bioc/html/rhdf5.html)
- [biomaRt](https://bioconductor.org/packages/release/bioc/html/biomaRt.html)
- [sva](https://www.bioconductor.org/packages/release/bioc/html/sva.html)
- [GAGE](https://bioconductor.org/packages/release/bioc/html/gage.html)

### Sleuth
[Source](https://pachterlab.github.io/sleuth/download)

devtools need to be installed if not already ```install.packages("devtools")```

install with ```devtools::install_github("pachterlab/sleuth")```


***
# General notes on RNA-seq workflow

Due to inconsistencies in the naming conventions for chromosomes, you should obtain your reference genome and annotation packages from the same source (Ensembl, NCBI, UCSC). Additionally, your annotations must correspond with the same reference genome version (i.e. both correspond to NCBI human GRCh38).

[This guide](https://pmbio.org/module-02-inputs/0002/02/01/Reference_Genome/) explains reference genomes and how to obtain them.



***
# Obtain known gene/transcript annotations

Download the annotations from Ensembl via ```wget http://genomedata.org/rnaseq-tutorial/annotations/GRCh38/chr22_with_ERCC92.gtf```. 

You can preview some of the file with ```cat chr22_with_ERCC92.gtf | column -t | less -p exon -S```.


```cat chr22_with_ERCC92.gtf | grep -w gene | wc -l```

You can view the structure of one transcript with ```grep ENST00000342247 $RNA_REF_GTF | less -p "exon\s" -S```. Exit with q.

Other sources of annontation files for a HISAT2/StringTie/Ballgown pipeline, refer to [here](https://rnabio.org/module-01-inputs/0001/03/01/Annotations/).

HISAT2 has prebuilt indexes for DNA and RNA for both Ensembl and UCSC genomes [here](https://ccb.jhu.edu/software/hisat2/index.shtml).

**Important**: chromosome names in the .gtf file must be the same as those in the reference genome .fa file. If StringTie returns expression values that are all 0, this is the likley cause of that error. Download the annotation and reference genome files from the same source.

**Make sure your reference genome and annotations are for the same build of the genome (i.e. both for GRCh38).**

***
# Creating index with HISAT2

This was done directly using python files from the [HISAT2 github repo](https://github.com/DaehwanKimLab/hisat2), not sure how to do it using the installed program listed above or if it's possible. Just clone the repo and use the files necessary.

Run the following commands from the location of your reference .gtf and .fa files. The chr22_with_ERCC92.gtf and chr22_with_ERCC92.fa. You may need a ```./``` in front of the first two commands

```hisat2_extract_splice_sites.py chr22_with_ERCC92.gtf > splicesites.tsv```

```hisat2_extract_exons.py chr22_with_ERCC92.gtf >exons.tsv```

```hisat2-build -p 8 --ss splicesites.tsv --exon exons.tsv chr22_with_ERCC92.fa chr22_with_ERCC92```

The output should be a collection of binary files with names like chr22_with_ERCC92.x.ht2, the x being a number.

***
# RNA-seq test data

```mkdir test_data && cd test_data```

```wget http://genomedata.org/rnaseq-tutorial/HBR_UHR_ERCC_ds_5pc.tar```

unpack the tarball

```tar -xvf HBR_UHR_ERCC_ds_5pc.tar```

Look at the head of one of the files.

```zcat UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz | head -n 8```

Determine how many reads are in the first library.

```zcat UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz | grep -P "^\@HWI" | wc -l```


***
# Check strandedness (optional, but not really)

This step is not required but due to differences in usage of RNA-seq library construction kits, it is **highly** recommended to empirically verify strandedness using the below method.

The check_strandedness should already be installed at the beginning. It can also be installed as a [Docker image](https://hub.docker.com/r/mgibio/checkstrandedness).

Convert gtf to genepred

```gtfToGenePred chr22_with_ERCC92.gtf chr22_with_ERCC92.genePred```

Convert genPred to bed12

```genePredToBed chr22_with_ERCC92.genePred chr22_with_ERCC92.bed12```

Use bedtools to create fasta from GTF

```bedtools getfasta -fi chr22_with_ERCC92.fa -bed chr22_with_ERCC92.bed12 -s -split -name -fo chr22_ERCC92.transcripts.fa```

Clean up the header lines of the transcript file

```cat chr22_ERCC92_transcripts.fa | perl -ne 'if($_ =~/^\>\S+\:\:(ERCC\-\d+)\:.*/){print ">$1\n"}elsif ($_ =~/^\>(\S+)\:\:.*/){print ">$1\n"}else{print $_}' > chr22_ERCC92_transcripts.clean.fa```

View the clean file

```less chr22_ERCC92_transcripts.clean.fa```

Add in transcript_id field to .gtf file

```awk '{ if ($0 ~ "transcript_id") print $0; else print $0" transcript_id \"\";"; }' chr22_with_ERCC92.gtf > chr22_with_ERCC92_tidy.gtf```

check strandedness 

```check_strandedness --gtf chr22_with_ERCC92_tidy.gtf --transcripts chr22_ERCC92_transcripts.clean.fa --reads_1 test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads_2 test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz```

OUTPUT:

Fraction of reads failed to determine: 0.1123

Fraction of reads explained by "1++,1--,2+-,2-+": 0.0155 (1.7% of explainable reads)

Fraction of reads explained by "1+-,1-+,2++,2--": 0.8722 (98.3% of explainable reads)

Over 90% of reads explained by "1+-,1-+,2++,2--"

Data is likely RF/fr-firststrand

More info on strandedness can be found [here](https://rnabio.org/module-09-appendix/0009/12/01/StrandSettings/)

***
# Pre-alignment QC

To get a sense of the raw reads quality in your fastq files, run the following programs.

```fastqc *.fastq.gz``` in the same directory as your reads.

It will output an HTML file for each read in the same directory that contains a full report of the quality measures.

In the "over-represented sequences" section, try [BLASTing](https://blast.ncbi.nlm.nih.gov/Blast.cgi) the sequences to try determining a source organism.

Run multiqc to combine all the fastqc reports.

```multiqc .``` run in the same directory as the reports.

Make a new directory for the fastqc .html outputs and move them

```mkdir fastqc && mv *_fastqc* fastqc```

***
# Trim adapters

Make a new folder for the trimmed reads

```mkdir trimmed```

Download the Illumina adapter sequence files, or the relevant files for your sequencing machine.

```wget http://genomedata.org/rnaseq-tutorial/illumina_multiplex.fa```

Use flexbar to remove adapter sequences and trim first 13 bases of each read. Adjust the number of threads to something manageable for your system. It is set to 24 threads and takes a few seconds to run for each command. It may take much longer with a slower CPU.

Possible options for flexbar and their meaning:

   - ```–adapter-min-overlap 7``` requires a minimum of 7 bases to match the adapter
   - ```–adapter-trim-end RIGHT``` uses a trimming strategy to remove the adapter from the 3 prime or RIGHT end of the read
   - ```–max-uncalled 300``` allows as many as 300 uncalled or N bases (MiSeq read lengths can be 300bp)
   - ```–min-read-length``` the minimum read length allowed after trimming is 25bp.
   - ```–threads 8``` use 8 threads
   - ```–zip-output GZ``` the input FASTQ files are gzipped so we will output gzipped FASTQ to save space
   - ```–adapters``` define the path to the adapter FASTA file to trim
   - ```–reads``` define the path to the read 1 FASTQ file of reads
   - ```–reads2``` define the path to the read 2 FASTQ file of reads
   - ```–target``` a base path for the output files. The value will _1.fastq.gz and _2.fastq.gz for read 1 and read 2 respectively
   - ```–pre-trim-left``` trim a fixed number of bases at left read end. For example, to trim 5 bases at the left side of reads: –pre-trim-left 5
   - ```–pre-trim-right``` trim a fixed number of bases at right read end. For example, to trim 5 bases at the right side of reads: –pre-trim-right 5
   - ```–pre-trim-phred``` trim based on phred quality value to deal with higher error rates towards the end of reads. For example, to trim the 3 prime end until quality offset value 30 or higher is reached, specify: –pre-trim-phred 30

The commands to run for flexbar for this dataset are as follows:

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22```

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22```

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22```

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22```

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22```

```./flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters /home/user/bioinformatics/ref_genome/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 24 --zip-output GZ --reads /home/user/bioinformatics/ref_genome/test_data/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target /home/user/bioinformatics/ref_genome/test_data/trimmed/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22```

Compare the impact of trimming with the original QC reports by running fastqc and multiqc on the trimmed reads

In the directory of the trimmed reads, run:

```fastqc *.fastq.gz```

```multiqc .```

Same as before, make a new directory for the fastqc .html outputs and move them

```mkdir fastqc && mv *_fastqc* fastqc```


***
# Alignment with HISAT2

For more details on HISAT2, refer to the [manual on Github](http://daehwankimlab.github.io/hisat2/manual/)

The outputs after running HISAT2 will be a SAM/BAM file for each data set.

Options for HISAT2 include:


   - ```-p 8``` tells HISAT2 to use eight threads for bowtie alignments.
   - ```–rna-strandness RF``` specifies strandness of RNAseq library. We will specify RF since the TruSeq strand-specific library was used to make these libraries. See here for options.
   - ```–rg-id $ID``` specifies a read group ID that is a unique identifier.
   - ```–rg SM:$SAMPLE_NAME``` specifies a read group sample name. This together with rg-id will allow you to determine which reads came from which sample in the merged bam later on.
   - ```–rg LB:$LIBRARY_NAME``` specifies a read group library name. This together with rg-id will allow you to determine which reads came from which library in the merged bam later on.
   - ```–rg PL:ILLUMINA``` specifies a read group sequencing platform.
   - ```–rg PU:$PLATFORM_UNIT``` specifies a read group sequencing platform unit. Typically this consists of FLOWCELL-BARCODE.LANE
   - ```–dta``` Reports alignments tailored for transcript assemblers.
   - ```-x /path/to/hisat2/index``` The HISAT2 index filename prefix (minus the trailing .X.ht2) built earlier including splice sites and exons.
   - ```-1 /path/to/read1.fastq.gz``` The read 1 FASTQ file, optionally gzip(.gz) or bzip2(.bz2) compressed.
   - ```-2 /path/to/read2.fastq.gz``` The read 2 FASTQ file, optionally gzip(.gz) or bzip2(.bz2) compressed.
   - ```-S /path/to/output.sam``` The output SAM format text file of alignments.

Run a command for each of your replicates.

```hisat2 -p 24 --rg-id=UHR_Rep1 --rg SM:UHR --rg LB:UHR_Rep1_ERCC-Mix1 --rg PL:ILLUMINA --rg PU:CXX1234-ACTGAC.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep1.sam```

```hisat2 -p 24 --rg-id=UHR_Rep2 --rg SM:UHR --rg LB:UHR_Rep2_ERCC-Mix1 --rg PL:ILLUMINA --rg PU:CXX1234-TGACAC.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep2.sam```

```hisat2 -p 24 --rg-id=UHR_Rep3 --rg SM:UHR --rg LB:UHR_Rep3_ERCC-Mix1 --rg PL:ILLUMINA --rg PU:CXX1234-CTGACA.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep3.sam```

```hisat2 -p 24 --rg-id=HBR_Rep1 --rg SM:HBR --rg LB:HBR_Rep1_ERCC-Mix2 --rg PL:ILLUMINA --rg PU:CXX1234-TGACAC.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep1.sam```

```hisat2 -p 24 --rg-id=HBR_Rep2 --rg SM:HBR --rg LB:HBR_Rep2_ERCC-Mix2 --rg PL:ILLUMINA --rg PU:CXX1234-GACACT.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep2.sam```

```hisat2 -p 24 --rg-id=HBR_Rep3 --rg SM:HBR --rg LB:HBR_Rep3_ERCC-Mix2 --rg PL:ILLUMINA --rg PU:CXX1234-ACACTG.1 -x /home/user/bioinformatics/ref_genome/chr22_with_ERCC92 --dta --rna-strandness RF -1 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz -2 /home/user/bioinformatics/ref_genome/test_data/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz -S /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep3.sam```

### Convert SAM to BAM files

```samtools sort -@ 8 -o UHR_Rep1.bam UHR_Rep1.sam```
```samtools sort -@ 8 -o UHR_Rep2.bam UHR_Rep2.sam```
```samtools sort -@ 8 -o UHR_Rep3.bam UHR_Rep3.sam```
```samtools sort -@ 8 -o HBR_Rep1.bam HBR_Rep1.sam```
```samtools sort -@ 8 -o HBR_Rep2.bam HBR_Rep2.sam```
```samtools sort -@ 8 -o HBR_Rep3.bam HBR_Rep3.sam```

### Merge all BAM files
Other tools such as "samtools merge" and "bamtools merge" can also achieve this. Create a BAM file for each group (UHR and HBR).

```java -Xmx2g -jar picard.jar MergeSamFiles -OUTPUT /home/user/bioinformatics/ref_genome/test_data/align_output/UHR.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep1.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep2.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep3.bam```

```java -Xmx2g -jar picard.jar MergeSamFiles -OUTPUT /home/user/bioinformatics/ref_genome/test_data/align_output/HBR.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep1.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep2.bam -INPUT /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep3.bam```

You can verify that all bam files were created successfully, there should be 8.

```ls -l *.bam | wc -l```

***
# Using the Integrative Genomics Viewer (IGV)

IGV can be run from the download directory using ```./igv.sh```

To re-visit the shortish tutorial on navigating IGV, go [here](https://rnabio.org/module-02-alignment/0002/04/01/IGV/). You can follow the tutorial there using the provided files.

For our dataset, we need to index the BAM files before loading them into IGV. This will be done with samtools. From the directory where the BAM files are located, run:

```samtools index HBR.bam```

```samtools index HBR_Rep1.bam```

```samtools index HBR_Rep2.bam```

```samtools index HBR_Rep3.bam```

```samtools index UHR.bam```

```samtools index UHR_Rep1.bam```

```samtools index UHR_Rep2.bam```

```samtools index UHR_Rep3.bam```

**Alternatively**, you can run the following command to index all .bam files in the directory. This way is easier than running multiple commands.

```find *.bam -exec echo samtools index {} \; | sh```

Load both the UHR.bam and HBR.bam files in IGV. Make sure the index file .bam.bai is in the same directory.

***
# BAM read counting

Make a new directory for the readcounts.

```mkdir bam_readcount && cd bam_readcount```

Create a faidx index for use with samtools mpileup. Place a copy of chr22_with_ERCC92.fa in the same directory below.

```samtools faidx /home/user/bioinformatics/ref_genome/test_data/bam_readcount/chr22_with_ERCC92.fa```

Run mpileup on a region of interest (ROI).

```samtools mpileup -f /home/user/bioinformatics/ref_genome/test_data/bam_readcount/chr22_with_ERCC92.fa -r 22:18918457-18918467 /home/user/bioinformatics/ref_genome/test_data/align_output/UHR.bam /home/user/bioinformatics/ref_genome/test_data/align_output/HBR.bam```

The output will only exist in the terminal. As per the tutorial, "Each line consists of chromosome, 1-based coordinate, reference base, the number of reads covering the site, read bases and base qualities. At the read base column, a dot stands for a match to the reference base on the forward strand, a comma for a match on the reverse strand, ```ACGTN``` for a mismatch on the forward strand and ```acgtn``` for a mismatch on the reverse strand. A pattern ```\+[0-9]+[ACGTNacgtn]+``` indicates there is an insertion between this reference position and the next reference position. The length of the insertion is given by the integer in the pattern, followed by the inserted sequence. [mpileup docs](http://samtools.sourceforge.net/mpileup.shtml).

In the ```bam_readcount``` directory, create a bed file with the variant position on the desired chromosome.

```echo "22 38483683 38483683" > snvs.bed```

Now we can run ```bam-readcount``` on both bam files

```./bam-readcount -l /home/user/bioinformatics/ref_genome/test_data/bam_readcount/snvs.bed -f /home/user/bioinformatics/ref_genome/test_data/bam_readcount/chr22_with_ERCC92.fa /home/user/bioinformatics/ref_genome/test_data/align_output/UHR.bam 2>/dev/null 1>/home/user/bioinformatics/ref_genome/test_data/bam_readcount/UHR_bam-readcounts.txt```

```./bam-readcount -l /home/user/bioinformatics/ref_genome/test_data/bam_readcount/snvs.bed -f /home/user/bioinformatics/ref_genome/test_data/bam_readcount/chr22_with_ERCC92.fa /home/user/bioinformatics/ref_genome/test_data/align_output/HBR.bam 2>/dev/null 1>/home/user/bioinformatics/ref_genome/test_data/bam_readcount/HBR_bam-readcounts.txt```

You can pipe the readcounts file to a perl script to parse the read counts for each base.

```cat UHR_bam-readcounts.txt | perl -ne '@data=split("\t", $_); @Adata=split(":", $data[5]); @Cdata=split(":", $data[6]); @Gdata=split(":", $data[7]); @Tdata=split(":", $data[8]); print "UHR Counts\t$data[0]\t$data[1]\tA: $Adata[1]\tC: $Cdata[1]\tT: $Tdata[1]\tG: $Gdata[1]\n";'```

```cat HBR_bam-readcounts.txt | perl -ne '@data=split("\t", $_); @Adata=split(":", $data[5]); @Cdata=split(":", $data[6]); @Gdata=split(":", $data[7]); @Tdata=split(":", $data[8]); print "HBR Counts\t$data[0]\t$data[1]\tA: $Adata[1]\tC: $Cdata[1]\tT: $Tdata[1]\tG: $Gdata[1]\n";'```

Outputs:
```UHR Counts      22      38483683        A: 163  C: 0    T: 0    G: 163```
```HBR Counts      22      38483683        A: 75   C: 0    T: 0    G: 131```

[Python tutorial for bam-readcount](https://github.com/genome/bam-readcount/tree/master/tutorial).

***
# Alignment QC

Use samtools to view the format of the alignment bam files.

```samtools view UHR.bam | head | column -t | less -S```
```samtools view HBR.bam | head | column -t | less -S```

You can add additional filtering flags to the samtools command. [This useful guide explains all of them](https://broadinstitute.github.io/picard/explain-flags.html). ```-f``` are required flags and ```-F``` are filtering flags.

```samtools view -f 3 -F 268 UHR.bam | head | column -t | less -S```

Here the ```-f 3``` requires the alignments are paired and 'mapped in a proper pair'. The ```-F 268``` means 'unmapped', 'mate is unmapped', and 'not primary alignment'.

Look for alignments that are only for 'PCR or optical duplicates'.

```samtools view -f 1024 UHR.bam | head```

Output: nothing

The command ```samtools flagstat``` can provide a summary of the alignments.

```mkdir flagstat```

```samtools flagstat HBR_Rep1.bam > flagstat/HBR_Rep1.bam.flagstat```

```samtools flagstat HBR_Rep2.bam > flagstat/HBR_Rep2.bam.flagstat```

```samtools flagstat HBR_Rep3.bam > flagstat/HBR_Rep3.bam.flagstat```

```samtools flagstat UHR_Rep1.bam > flagstat/UHR_Rep1.bam.flagstat```

```samtools flagstat UHR_Rep2.bam > flagstat/UHR_Rep2.bam.flagstat```

```samtools flagstat UHR_Rep3.bam > flagstat/UHR_Rep3.bam.flagstat```

In this instance, you can also run an expression to do it in one command.

```find *Rep*.bam -exec echo samtools flagstat {} \> flagstat/{}.flagstat \; | sh```

### FastQc on bam file

From your alignment directory (where the bam files are).

```mkidr fastqc```

```fastqc UHR_Rep1.bam UHR_Rep2.bam UHR_Rep3.bam HBR_Rep1.bam HBR_Rep2.bam HBR_Rep3.bam```

Clean up the current directory and move the reports into the fastqc folder.

```mv *fastqc.html fastqc/```

```mv *fastqc.zip fastqc/```

### Picard

Using Picard we can generate metrics and figures that express the quality of our RNA-seq data.

We first need to create input files for Picard based on the reference genome so it can properly run the function CollectRnaSeqMetrics.

```java -jar picard.jar CreateSequenceDictionary -R /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.fa -O /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.dict```

The next line is copied directly from the source tutorial.

"Create a bed file of the location of ribosomal sequences in our reference (first extract from the gtf then convert to bed). Note that here we pull all the "rrna" transcripts from the GTF. This is a good strategy for the whole transcriptome ... but on chr22 there is very little "rrna" content, leading to 0 coverage for all samples, so we are also adding a single protein coding ribosomal gene "RRP7A" (normally we would not do this)"

```grep --color=none -i -P "rrna|rrp7a" /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > /home/user/bioinformatics/ref_genome/ref_ribosome.gtf```

Note: ```gff2bed``` is part of BEDOPS. Run the script directly from the bedops/bin folder. You may need to add the /bin folder to your path variable ```export PATH="$PATH:/home/user/bioinformatics/bedops_linux_x86_64-v2.4.40/bin"```, otherwise convert2bed may not be found.

```gff2bed < /home/user/bioinformatics/ref_genome/ref_ribosome.gtf > /home/user/bioinformatics/ref_genome/ref_ribosome.bed```

Create interval list file to identify the location of the ribosomal sequences in the reference

```java -jar /home/user/bioinformatics/picard.jar BedToIntervalList -I /home/user/bioinformatics/ref_genome/ref_ribosome.bed -O /home/user/bioinformatics/ref_genome/ref_ribosome.interval_list -SD /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.dict```

Create genePred file for the reference transcriptome

```gtfToGenePred -genePredExt /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.ref_flat.txt```

Reformat the genePred file

```cat /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.ref_flat.txt | awk '{print $12"\t"$0}' | cut -d$'\t' -f1-11 > /home/user/bioinformatics/ref_genome/tmp.txt```

```mv /home/user/bioinformatics/ref_genome/tmp.txt /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.ref_flat.txt```

Go to the directory with your alignments.

```mkdir picard```

```find *Rep*.bam -exec echo java -jar /home/user/bioinformatics/picard.jar CollectRnaSeqMetrics I={} O=picard/{}.RNA_Metrics REF_FLAT=/home/user/bioinformatics/ref_genome/chr22_with_ERCC92.ref_flat.txt STRAND=SECOND_READ_TRANSCRIPTION_STRAND RIBOSOMAL_INTERVALS=/home/user/bioinformatics/ref_genome/ref_ribosome.interval_list \; | sh```

### RSeQC (optional)

Another tool for generating RNA-seq QC reports.

You will need
- aligned bam files
- index file for each bam file
- transcript bed file (in bed12 format)

Start by converting the reference gtf to genePred

```gtfToGenePred /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.genePred```

Then convert the genePred to bed12

```genePredToBed /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.genePred /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12```

Go back to the alignment directory. Make sure the bioinformatics env is active.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output && mkdir rseqc```

```geneBody_coverage.py -i UHR_Rep1.bam,UHR_Rep2.bam,UHR_Rep3.bam -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 -o rseqc/UHR```

```geneBody_coverage.py -i HBR_Rep1.bam,HBR_Rep2.bam,HBR_Rep3.bam -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 -o rseqc/HBR```

The following commands should also be run from the alignment directory ```cd /home/user/bioinformatics/ref_genome/test_data/align_output```

```find *Rep*.bam -exec echo inner_distance.py -i {} -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 -o rseqc/{} \; | sh```

```find *Rep*.bam -exec echo junction_annotation.py -i {} -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 -o rseqc/{} \; | sh```

```find *Rep*.bam -exec echo junction_saturation.py -i {} -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 -o rseqc/{} \; | sh```

```find *Rep*.bam -exec echo read_distribution.py  -i {} -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 \> rseqc/{}.read_dist.txt \; | sh```

```find *Rep*.bam -exec echo RNA_fragment_size.py -i {} -r /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.bed12 \> rseqc/{}.frag_size.txt \; | sh```

```find *Rep*.bam -exec echo bam_stat.py -i {} \> {}.bam_stat.txt \; | sh```

```rm -f log.txt```

### MultiQC

Now we will compile a multiQC report from all of the QC done above.

In the alignment directory ```cd /home/user/bioinformatics/ref_genome/test_data/align_output```.

```multiqc ./```

***
# Expression Analysis

Some notes before we start.

First, there are multipls things you can do with your aligned reads from HISAT2. This pipeline is written with the intention of going through the differential expression analysis and visualizing it. The tutorial course can go even further than that, which I may or may not include in this. But, you can also just use **HTSEQ-count** to get raw read counts instead of FPKM values. 

FPKM = fragments per kilobase of transcript per million mapped reads. FPKM attempts to normalize for gene size and library depth. RPKM is essentially the same thing, however there are differing opinions on just how similar they are, and which is better. 

Mathematically, FPKM can be represented as:

```FPKM = (C / (N / 1000000)) / (L / 1000)```, where C = number of mappable fragments for a gene, N = total number of mappable fragments in the library, and L = number of base pairs in the gene.

Another normalization metric that may be mentioned is TPM (transcript per kilobase million). The only difference between FPKM and TPM is the order of operations.

### Stringtie

If you want to know more about the math behind how Stringtie works, you can [read the paper](https://www.nature.com/articles/nbt.3122).

An outline of the Stringtie flags used:


- ```–rf``` tells StringTie that our data is stranded and to use the correct strand specific mode (i.e. assume a stranded library fr-firststrand).
- ```-p 8``` tells StringTie to use eight CPUs
- ```-G``` reference annotation to use for guiding the assembly process (GTF/GFF3)
- ```-e``` only estimate the abundance of given reference transcripts (requires -G)
- ```-B``` enable output of Ballgown table files which will be created in the same directory as the output GTF (requires -G, -o recommended)
- ```-o``` output path/file name for the assembled transcripts GTF (default: stdout)
- ```-A``` output path/file name for gene abundance estimates

I'll make this folder within the alignment folder ```cd /home/user/bioinformatics/ref_genome/test_data/align_output```.

```mkdir -p expression/stringtie/ref_only/ && cd expression/stringtie/ref_only/```

```./stringtie --rf -p 24 -G /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf -e -B -o /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep1/transcripts.gtf -A /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep1/gene_abundances.tsv /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep1.bam```

```./stringtie --rf -p 24 -G /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf -e -B -o /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep2/transcripts.gtf -A /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep2/gene_abundances.tsv /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep2.bam```

```./stringtie --rf -p 24 -G /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf -e -B -o /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep3/transcripts.gtf -A /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/HBR_Rep3/gene_abundances.tsv /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep3.bam```

```./stringtie --rf -p 24 -G /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf -e -B -o /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep1/transcripts.gtf -A /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep1/gene_abundances.tsv /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep1.bam```

```./stringtie --rf -p 24 -G /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf -e -B -o /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep3/transcripts.gtf -A /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep3/gene_abundances.tsv /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep3.bam```

Have a look at what the raw output from StringTie looks like.

```less -S /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep1/transcripts.gtf```

Improve formatting of the *less* output.

```grep -v "^#" /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep1/transcripts.gtf | grep -w "transcript" | column -t | less -S```

You can also limit the view to just the transcript records and the correspoding FPKM values.

```awk '{if ($3=="transcript") print}' /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/UHR_Rep1/transcripts.gtf | cut -f 1,4,9 | column -t | less -S```

You can also view the transcript level expression values for each set of reads in the two files ```t_data.ctab``` and ```gene_abundances.tsv```. They should be located in the folders ```~/expression/stringtie/ref_only/``` and then in the UHR or HBR folders.

Download the script for creating an expression matrix based off the Stringtie outputs.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only```

```wget https://raw.githubusercontent.com/griffithlab/rnabio.org/master/assets/scripts/stringtie_expression_matrix.pl```

```chmod +x stringtie_expression_matrix.pl```

```./stringtie_expression_matrix.pl --expression_metric=TPM --result_dirs='HBR_Rep1,HBR_Rep2,HBR_Rep3,UHR_Rep1,UHR_Rep2,UHR_Rep3' --transcript_matrix_file=transcript_tpm_all_samples.tsv --gene_matrix_file=gene_tpm_all_samples.tsv```

```./stringtie_expression_matrix.pl --expression_metric=FPKM --result_dirs='HBR_Rep1,HBR_Rep2,HBR_Rep3,UHR_Rep1,UHR_Rep2,UHR_Rep3' --transcript_matrix_file=transcript_fpkm_all_samples.tsv --gene_matrix_file=gene_fpkm_all_samples.tsv```

```./stringtie_expression_matrix.pl --expression_metric=Coverage --result_dirs='HBR_Rep1,HBR_Rep2,HBR_Rep3,UHR_Rep1,UHR_Rep2,UHR_Rep3' --transcript_matrix_file=transcript_coverage_all_samples.tsv --gene_matrix_file=gene_coverage_all_samples.tsv```

View the TPM output files.

```column -t transcript_tpm_all_samples.tsv | less -S```

```column -t gene_tpm_all_samples.tsv | less -S```

### Generate raw counts instead of FPKM/TPM values

If you only want to generate the raw counts this can be done using HTSeq-count [docs](https://htseq.readthedocs.io/).

Some notes about the flags used:


- ```–format``` specify the input file format one of BAM or SAM. Since we have BAM format files, select ‘bam’ for this option.
- ```–order``` provide the expected sort order of the input file. Previously we generated position sorted BAM files so use ‘pos’.
- ```–mode``` determines how to deal with reads that overlap more than one feature. We believe the ‘intersection-strict’ mode is best.
- ```–stranded``` specifies whether data is stranded or not. The TruSeq strand-specific RNA libraries suggest the ‘reverse’ option for this parameter.
- ```–minaqual``` will skip all reads with alignment quality lower than the given minimum value
- ```–type``` specifies the feature type (3rd column in GFF file) to be used. (default, suitable for RNA-Seq and Ensembl GTF files: exon)
- ```–idattr``` The feature ID used to identify the counts in the output table. The default, suitable for RNA-SEq and Ensembl GTF files, is gene_id.

Change directories to whre the expression folder is ```cd /home/user/bioinformatics/ref_genome/test_data/align_output```

```mkdir -p expression/htseq_counts && cd expression/htseq_counts```

Run each htseq-count command to get the raw counts for each read file.

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep1.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > UHR_Rep1_gene.tsv```

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep2.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > UHR_Rep2_gene.tsv```

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/UHR_Rep3.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > UHR_Rep3_gene.tsv```

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep1.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > HBR_Rep1_gene.tsv```

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep2.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > HBR_Rep2_gene.tsv```

```htseq-count --format bam --order pos --mode intersection-strict --stranded reverse --minaqual 1 --type exon --idattr gene_id /home/user/bioinformatics/ref_genome/test_data/align_output/HBR_Rep3.bam /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf > HBR_Rep3_gene.tsv```

I don't feel like paraphrasing this next section:

"Merge results files into a single matrix for use in edgeR. The following joins the results for each replicate together, adds a header, reformats the result as a tab delimited file, and shows you the first 10 lines of the resulting file :"

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/expression/htseq_counts```

```echo "GeneID UHR_Rep1 UHR_Rep2 UHR_Rep3 HBR_Rep1 HBR_Rep2 HBR_Rep3" > header.txtne.tsv | join - HBR_Rep2_gene.tsv | join - HBR_Rep3_gene.tsv > gene_read_counts_table_all.tsv```

```echo "GeneID UHR_Rep1 UHR_Rep2 UHR_Rep3 HBR_Rep1 HBR_Rep2 HBR_Rep3" > header.txt```

```cat header.txt gene_read_counts_table_all.tsv | grep -v "__" | awk -v OFS="\t" '$1=$1' > gene_read_counts_table_all_final.tsv```

If you're curious, ```grep -v "__"``` is used to filter out summary lines at the end of htseq-count files that did not align, had poor quality alignment, etc.

```rm -f gene_read_counts_table_all.tsv header.txt```

```head gene_read_counts_table_all_final.tsv | column -t```

### ERCC expression analysis

I'm not sure how necessary this next part is at the moment, but since the files used in this version of the pipeline included ERCC spike-in controls I will go through it.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/expression/htseq_counts```

```wget http://genomedata.org/rnaseq-tutorial/ERCC_Controls_Analysis.txt```

```cat ERCC_Controls_Analysis.txt```

```wget https://github.com/griffithlab/rnabio.org/raw/master/assets/scripts/Tutorial_ERCC_expression.pl```

```chmod +x Tutorial_ERCC_expression.pl```

```./Tutorial_ERCC_expression.pl```

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/expression/htseq_counts/ercc_read_counts.tsv```

```wget https://github.com/griffithlab/rnabio.org/raw/master/assets/scripts/Tutorial_ERCC_expression.R```

```chmod +x Tutorial_ERCC_expression.R```

```./Tutorial_ERCC_expression.R ercc_read_counts.tsv```

Create plot for ERCC spike-in Stringtie TPM estimates vs. known concentration of ERCC spike-in mix.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only```

```wget http://genomedata.org/rnaseq-tutorial/ERCC_Controls_Analysis.txt```

```cat ERCC_Controls_Analysis.txt```

```wget https://github.com/griffithlab/rnabio.org/raw/master/assets/scripts/Tutorial_ERCC_expression_tpm.pl```

```chmod +x Tutorial_ERCC_expression_tpm.pl```

```./Tutorial_ERCC_expression_tpm.pl```

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/ercc_tpm.tsv```

```wget https://github.com/griffithlab/rnabio.org/raw/master/assets/scripts/Tutorial_ERCC_expression_tpm.R```

```chmod +x Tutorial_ERCC_expression_tpm.R```

```./Tutorial_ERCC_expression_tpm.R ercc_tpm.tsv```

You should be able to view the generated figures under the ```~/htseq_counts``` and ```~/stringtie/ref_only``` directories. The figures will be in .pdf format.

***
# Differential Expression

The differential expression is done using the bioconductor package [Ballgown](https://www.bioconductor.org/packages/release/bioc/html/ballgown.html).

```cd /home/user/bioinformatics/ref_genome/test_data/align_output```

```mkdir -p de/ballgown/ref_only/ && cd de/ballgown/ref_only/```

Start an R session by typing ```R``` in a terminal.

The following code is an R script that will perform DE using Ballgown. You can run each command line by line.

In [2]:
# # load the required libraries
# library(ballgown)
# library(genefilter)
# library(dplyr)
# library(devtools)

# # Create phenotype data needed for ballgown analysis
# ids=c("UHR_Rep1","UHR_Rep2","UHR_Rep3","HBR_Rep1","HBR_Rep2","HBR_Rep3")
# type=c("UHR","UHR","UHR","HBR","HBR","HBR")
# results="/home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/"
# path=paste(results,ids,sep="")
# pheno_data=data.frame(ids,type,path)

# # Load ballgown data structure and save it to a variable "bg"
# bg = ballgown(samples=as.vector(pheno_data$path), pData=pheno_data)

# # Display a description of this object
# bg

# # Load all attributes including gene name
# bg_table = texpr(bg, 'all')
# bg_gene_names = unique(bg_table[, 9:10])

# # Save the ballgown object to a file for later use
# save(bg, file='bg.rda')

# # Perform differential expression (DE) analysis with no filtering
# results_transcripts = stattest(bg, feature="transcript", covariate="type", getFC=TRUE, meas="FPKM")
# results_genes = stattest(bg, feature="gene", covariate="type", getFC=TRUE, meas="FPKM")
# results_genes = merge(results_genes, bg_gene_names, by.x=c("id"), by.y=c("gene_id"))

# # Save a tab delimited file for both the transcript and gene results
# write.table(results_transcripts, "UHR_vs_HBR_transcript_results.tsv", sep="\t", quote=FALSE, row.names = FALSE)
# write.table(results_genes, "UHR_vs_HBR_gene_results.tsv", sep="\t", quote=FALSE, row.names = FALSE)

# # Filter low-abundance genes. Here we remove all transcripts with a variance across the samples of less than one
# bg_filt = subset (bg,"rowVars(texpr(bg)) > 1", genomesubset=TRUE)

# # Load all attributes including gene name
# bg_filt_table = texpr(bg_filt , 'all')
# bg_filt_gene_names = unique(bg_filt_table[, 9:10])

# # Perform DE analysis now using the filtered data
# results_transcripts = stattest(bg_filt, feature="transcript", covariate="type", getFC=TRUE, meas="FPKM")
# results_genes = stattest(bg_filt, feature="gene", covariate="type", getFC=TRUE, meas="FPKM")
# results_genes = merge(results_genes, bg_filt_gene_names, by.x=c("id"), by.y=c("gene_id"))

# # Output the filtered list of genes and transcripts and save to tab delimited files
# write.table(results_transcripts, "UHR_vs_HBR_transcript_results_filtered.tsv", sep="\t", quote=FALSE, row.names = FALSE)
# write.table(results_genes, "UHR_vs_HBR_gene_results_filtered.tsv", sep="\t", quote=FALSE, row.names = FALSE)

# # Identify the significant genes with p-value < 0.05
# sig_transcripts = subset(results_transcripts, results_transcripts$pval<0.05)
# sig_genes = subset(results_genes, results_genes$pval<0.05)

# # Output the significant gene results to a pair of tab delimited files
# write.table(sig_transcripts, "UHR_vs_HBR_transcript_results_sig.tsv", sep="\t", quote=FALSE, row.names = FALSE)
# write.table(sig_genes, "UHR_vs_HBR_gene_results_sig.tsv", sep="\t", quote=FALSE, row.names = FALSE)

# # Exit the R session
# quit(save="no")

View the raw output from Ballgown.

```head UHR_vs_HBR_gene_results.tsv | column -t```

View how many genes are present on this chromosome.

```grep -v feature UHR_vs_HBR_gene_results.tsv | wc -l```

View the genes that passed the filtering step.

```grep -v feature UHR_vs_HBR_gene_results_filtered.tsv | wc -l```

View how many genes were considered differentially expressed (p < 0.05).

```grep -v feature UHR_vs_HBR_gene_results_sig.tsv | wc -l```

As an exercise or just a visualization, you can look at the top 20 DE genes in IGV and see if they seem to make sense. To explain the 

```grep -v feature UHR_vs_HBR_gene_results_sig.tsv | sort -rnk 3 | head -n 20 | column -t``` higher abundance in UHR

```grep -v feature UHR_vs_HBR_gene_results_sig.tsv | sort -nk 3 | head -n 20 | column -t``` higher abundance in HBR

Save all the genes that have a p < 0.05.

```grep -v feature UHR_vs_HBR_gene_results_sig.tsv | cut -f 6 | sed 's/\"//g' > DE_genes.txt```

```head DE_genes.txt```

### ERCC DE analysis

Also not sure if this section is necessary and how it will be done in other pipelines analyzing data using different spike-ins.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only```

```wget https://raw.githubusercontent.com/griffithlab/rnabio.org/master/assets/scripts/Tutorial_ERCC_DE.R```

```chmod +x Tutorial_ERCC_DE.R```

```./Tutorial_ERCC_DE.R /home/user/bioinformatics/ref_genome/test_data/align_output/expression/htseq_counts/ERCC_Controls_Analysis.txt /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only/UHR_vs_HBR_gene_results.tsv```

The results are output to a PDF file.

***
# edgeR analysis

This is an alternative method of doing DE to the stringtie/ballgown method. This method uses the raw counts from htseq-count instead of the FPKM values.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/de```

```mkdir -p htseq_counts && cd htseq_counts```

htseq-count only proivdes the Ensembl Gene ID for the counts and not the actual gene symbol. This next line attempts to do that using the GTF file for our transcriptome reference.

```perl -ne 'if ($_ =~ /gene_id\s\"(ENSG\S+)\"\;/) { $id = $1; $name = undef; if ($_ =~ /gene_name\s\"(\S+)"\;/) { $name = $1; }; }; if ($id && $name) {print "$id\t$name\n";} if ($_=~/gene_id\s\"(ERCC\S+)\"/){print "$1\t$1\n";}' /home/user/bioinformatics/ref_genome/chr22_with_ERCC92.gtf | sort | uniq > ENSG_ID2Name.txt```

```head ENSG_ID2Name.txt```

Count the unique gene ids.

```cut -f 1 ENSG_ID2Name.txt | sort | uniq | wc -l```

Count the unique gene names.

```cut -f 2 ENSG_ID2Name.txt | sort | uniq | wc -l```

Look at the most repeatd gene names.

```cut -f 2 ENSG_ID2Name.txt | sort | uniq -c | sort -r | head```

Run the R code in the cell below, uncomment everything first with ctrl + /.

In [5]:
# #Set working directory where output will go
# working_dir = "/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts"
# setwd(working_dir)

# #Read in gene mapping
# mapping=read.table("/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/ENSG_ID2Name.txt", header=FALSE, stringsAsFactors=FALSE, row.names=1)

# # Read in count matrix
# rawdata=read.table("/home/user/bioinformatics/ref_genome/test_data/align_output/expression/htseq_counts/gene_read_counts_table_all_final.tsv", header=TRUE, stringsAsFactors=FALSE, row.names=1)

# # Check dimensions
# dim(rawdata)

# # Require at least 1/6 of samples to have expressed count >= 10
# sample_cutoff <- (1/6)
# count_cutoff <- 10

# #Define a function to calculate the fraction of values expressed above the count cutoff
# getFE <- function(data,count_cutoff){
#  FE <- (sum(data >= count_cutoff)/length(data))
#  return(FE)
# }

# #Apply the function to all genes, and filter out genes not meeting the sample cutoff
# fraction_expressed <- apply(rawdata, 1, getFE, count_cutoff)
# keep <- which(fraction_expressed >= sample_cutoff)
# rawdata <- rawdata[keep,]

# # Check dimensions again to see effect of filtering
# dim(rawdata)

# #################
# # Running edgeR #
# #################

# # load edgeR
# library('edgeR')

# # make class labels
# class <- c( rep("UHR",3), rep("HBR",3) )

# # Get common gene names
# Gene=rownames(rawdata)
# Symbol=mapping[Gene,1]
# gene_annotations=cbind(Gene,Symbol)

# # Make DGEList object
# y <- DGEList(counts=rawdata, genes=gene_annotations, group=class)
# nrow(y)

# # TMM Normalization
# y <- calcNormFactors(y)

# # Estimate dispersion
# y <- estimateCommonDisp(y, verbose=TRUE)
# y <- estimateTagwiseDisp(y)

# # Differential expression test
# et <- exactTest(y)

# # Extract raw counts to add back onto DE results
# counts <- getCounts(y)

# # Print top genes
# topTags(et)

# # Print number of up/down significant genes at FDR = 0.05  significance level
# summary(de <- decideTestsDGE(et, adjust.method="BH", p=.05))

# #Get output with BH-adjusted FDR values - all genes, any p-value, unsorted
# out <- topTags(et, n = "Inf", adjust.method="BH", sort.by="none", p.value=1)$table

# #Add raw counts back onto results for convenience (make sure sort and total number of elements allows proper join)
# out2 <- cbind(out, counts)

# #Limit to significantly DE genes
# out3 <- out2[as.logical(de),]

# # Order by p-value
# o <- order(et$table$PValue[as.logical(de)],decreasing=FALSE)
# out4 <- out3[o,]

# # Save table
# write.table(out4, file="DE_genes.txt", quote=FALSE, row.names=FALSE, sep="\t")

***
Compare the significantly DE genes between the 2 methods.

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only/DE_genes.txt```

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/DE_genes.txt```

Pull out the gene IDs from both files.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/de```

```cut -f 1 /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only/DE_genes.txt | sort  > ballgown_DE_gene_symbols.txt```

```cut -f 2 /home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/DE_genes.txt | sort | uniq | grep -v Gene_Name > htseq_counts_edgeR_DE_gene_symbols.txt```

You can use online tools to quickly create a venn diagram comparing the DE genes from the two methods. You can use the 2 cat commands, then copy/paste the outputs to the websites. Or just open the .txt files and copy from there. [biovenn.nl](https://www.biovenn.nl/) or [venny](https://bioinfogp.cnb.csic.es/tools/venny/).

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown_DE_gene_symbols.txt```

```cat /home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts_edgeR_DE_gene_symbols.txt```

***
# Ballgown DE Visualization

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only```

Run the R code in the cell below. Uncomment all the lines with ctrl + / if needed.

In [8]:
# #load libraries
# library(ballgown)
# library(genefilter)
# library(dplyr)
# library(devtools)

# #designate output file
# outfile="/home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only/Tutorial_Part2_ballgown_output.pdf"

# # Generate phenotype data
# ids = c("UHR_Rep1","UHR_Rep2","UHR_Rep3","HBR_Rep1","HBR_Rep2","HBR_Rep3")
# type = c("UHR","UHR","UHR","HBR","HBR","HBR")
# results = "/home/user/bioinformatics/ref_genome/test_data/align_output/expression/stringtie/ref_only/"
# path = paste(results,ids,sep="")
# pheno_data = data.frame(ids,type,path)

# # Display the phenotype data
# pheno_data

# # Load the ballgown object from file
# # Make sure to set the path to 'bg.rda', this file was created in the ballgown DE R script
# load('/home/user/bioinformatics/ref_genome/test_data/align_output/de/ballgown/ref_only/bg.rda')

# # The load command, loads an R object from a file into memory in our R session.
# # You can use ls() to view the names of variables that have been loaded
# ls()

# # Print a summary of the ballgown object
# bg

# # Open a PDF file where we will save some plots. We will save all figures and then view the PDF at the end
# pdf(file=outfile)

# # Extract FPKM values from the 'bg' object
# fpkm = texpr(bg,meas="FPKM")

# # View the last several rows of the FPKM table
# tail(fpkm)

# # Transform the FPKM values by adding 1 and convert to a log2 scale
# fpkm = log2(fpkm+1)

# # View the last several rows of the transformed FPKM table
# tail(fpkm)

# # Create boxplots to display summary statistics for the FPKM values for each sample
# boxplot(fpkm,col=as.numeric(as.factor(pheno_data$type))+1,las=2,ylab='log2(FPKM+1)')

# # col=as.numeric(as.factor(pheno_data$type))+1 - set color based on pheno_data$type which is UHR vs. HBR
# # las=2 - labels are perpendicular to axis 
# # ylab='log2(FPKM+1)' - set ylab to indicate that values are log2 transformed

# # Display the transcript ID for a single row of data
# ballgown::transcriptNames(bg)[2763]

# # Display the gene name for a single row of data
# ballgown::geneNames(bg)[2763]

# # Create a BoxPlot comparing the expression of a single gene for all replicates of both conditions
# boxplot(fpkm[2763,] ~ pheno_data$type, border=c(2,3), main=paste(ballgown::geneNames(bg)[2763],' : ', ballgown::transcriptNames(bg)[2763]),pch=19, xlab="Type", ylab='log2(FPKM+1)')

# # border=c(2,3) - set border color for each of the boxplots
# # main=paste(ballgown::geneNames(bg)[2763],' : ', ballgown::transcriptNames(bg)[2763]) - set title to gene : transcript
# # xlab="Type" - set x label to Type
# # ylab='log2(FPKM+1)' - set ylab to indicate that values are log2 transformed


# # Add the FPKM values for each sample onto the plot
# points(fpkm[2763,] ~ jitter(c(2,2,2,1,1,1)), col=c(2,2,2,1,1,1)+1, pch=16)
# # pch=16 - set plot symbol to solid circle, default is empty circle


# # Create a plot of transcript structures observed in each replicate and color transcripts by expression level
# plotTranscripts(ballgown::geneIDs(bg)[2763], bg, main=c('TST in all HBR samples'), sample=c('HBR_Rep1', 'HBR_Rep2', 'HBR_Rep3'), labelTranscripts=TRUE)
# plotTranscripts(ballgown::geneIDs(bg)[2763], bg, main=c('TST in all UHR samples'), sample=c('UHR_Rep1', 'UHR_Rep2', 'UHR_Rep3'), labelTranscripts=TRUE)

# #plotMeans('TST',bg,groupvar="type",legend=FALSE)

# # Close the PDF device where we have been saving our plots
# dev.off()

***
# Batch Correction

The dataset used for everything leading up to this step was done at a single center, at the same time, using the same methodology. Therefore batch effects are not expected. A different dataset will be used to demonstrate this process.

It is highly recommended to read the entire [ComBat-seq paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7518324/) to better understand the concept of batch correction, and how it compares to normalization. **Cite the paper if this methodology is used**.

Read more about the reasoning behind this step on the [rnabio website](https://rnabio.org/module-03-expression/0003/05/01/Batch-Correction/).

```cd /home/user/bioinformatics```

```mkdir batch_correction && cd batch_correction```

```wget http://genomedata.org/rnaseq-tutorial/batch_correction/GSE48035_ILMN.counts.txt.gz```

Now create a version of the file that only has counts for the samples we need for this analysis.

Remove all quotes from the file

```zcat GSE48035_ILMN.counts.txt.gz | tr -d '"' > GSE48035_ILMN.counts.tmp.txt```

Create a fixed version of the header

```head -n 1 GSE48035_ILMN.counts.tmp.txt | perl -ne 'print "Gene\tChr\t$_"' > header.txt```

Split the chromosome and gene names on each line, sort the file by gene name.

```perl -ne 'chomp; if ($_ =~ /^(chr\w+)\!(\S+)(.*)/){print "$2\t$1$3\n"}else{print "$_\n"}' GSE48035_ILMN.counts.tmp.txt | sort > GSE48035_ILMN.counts.tmp2.txt```

Remove old header line.

```grep -v --color=never ABRF GSE48035_ILMN.counts.tmp2.txt > GSE48035_ILMN.counts.clean.txt```

Cut out columns for the UHR (A) and HBR (B) samples, replicates 1-4, and PolyA vs Enrichment.

```cut -f 1-2,3-6,7-10,19-22,23-26 GSE48035_ILMN.counts.clean.txt > GSE48035_ILMN.Counts.SampleSubset.txt```

```cut -f 1-2,3-6,7-10,19-22,23-26 header.txt > header.SampleSubset.txt```

Count the number of gene lines.

```wc -l GSE48035_ILMN.Counts.SampleSubset.txt```

Remove tmp files.

```rm -f GSE48035_ILMN.counts.txt.gz GSE48035_ILMN.counts.tmp.txt GSE48035_ILMN.counts.tmp2.txt GSE48035_ILMN.counts.clean.txt header.txt```

Now we want to limit the counts to only those that correspond to known protein coding genes.

Download the complete Ensembl GTF file

```wget ftp://ftp.ensembl.org/pub/release-101/gtf/homo_sapiens/Homo_sapiens.GRCh38.101.gtf.gz```

Grab all the gene records, limit to gene with "protein_coding" biotype, create unique gene name list.

```zcat Homo_sapiens.GRCh38.101.gtf.gz | grep -w gene | grep "gene_biotype \"protein_coding\"" | cut -f 9 | cut -d ";" -f 3 | tr -d " gene_name " | tr -d '"' | sort | uniq > Ensembl101_ProteinCodingGeneNames.txt```

Count unique protein coding genes.

```wc -l Ensembl101_ProteinCodingGeneNames.txt```

Filter gene counts down to only protein coding ones.

```join -j 1 -t $'\t' Ensembl101_ProteinCodingGeneNames.txt GSE48035_ILMN.Counts.SampleSubset.txt | cat header.SampleSubset.txt - > GSE48035_ILMN.Counts.SampleSubset.ProteinCodingGenes.tsv```

How many lines of RNA-seq count still present

```wc -l GSE48035_ILMN.Counts.SampleSubset.ProteinCodingGenes.tsv```

Remove tmp files.

```rm -f header.SampleSubset.txt GSE48035_ILMN.Counts.SampleSubset.txt```

View final read count matrix.

```column -t GSE48035_ILMN.Counts.SampleSubset.ProteinCodingGenes.tsv | less -S```

**Note about filtering**: Filtering by gene name is not ideal, as they may be incompatible between lists. It is better to use a unique identifier.

### Principal component analysis

Run the R code in the cell below. Uncomment all lines with ctrl + / if needed.

In [11]:
#load neccessary libraries
library("sva") #Note this exercise requires sva (>= v3.36.0) which is only available for R (>= v4.x)
library("ggplot2")
library("gridExtra")
library("edgeR")
library("UpSetR")

#load in the uncorrected data as raw counts
uncorrected_data = read.table("/home/user/bioinformatics/batch_correction/GSE48035_ILMN.Counts.SampleSubset.ProteinCodingGenes.tsv", header=TRUE, sep="\t", as.is=c(1,2))

#simplify the names of the data columns
# (A = Universal Human Reference RNA and B = Human Brain Reference RNA)
# RNA = polyA enrichment and RIBO = ribosomal RNA depletion
# 1, 2, 3, 4 are replicates
names(uncorrected_data) = c("Gene", "Chr", "UHR_Ribo_1", "UHR_Ribo_2", "UHR_Ribo_3", "UHR_Ribo_4", "HBR_Ribo_1", "HBR_Ribo_2", "HBR_Ribo_3", "HBR_Ribo_4", 
                            "UHR_Poly_1", "UHR_Poly_2", "UHR_Poly_3", "UHR_Poly_4", "HBR_Poly_1", "HBR_Poly_2", "HBR_Poly_3", "HBR_Poly_4")
sample_names = names(uncorrected_data)[3:length(names(uncorrected_data))]

#review data structure
head(uncorrected_data)
dim(uncorrected_data)

#define conditions, library methods, and replicates
conditions = c("UHR", "UHR", "UHR", "UHR", "HBR", "HBR", "HBR", "HBR", "UHR", "UHR", "UHR", "UHR", "HBR", "HBR", "HBR", "HBR")
library_methods = c("Ribo", "Ribo", "Ribo", "Ribo", "Ribo", "Ribo", "Ribo", "Ribo", "Poly", "Poly", "Poly", "Poly", "Poly", "Poly", "Poly", "Poly")
replicates = c(1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4)

#calculate principal components for the uncorrected data
pca_uncorrected_obj = prcomp(uncorrected_data[,sample_names])

#pull PCA values out of the PCA object
pca_uncorrected = as.data.frame(pca_uncorrected_obj[2]$rotation)

#assign labels to the data frame
pca_uncorrected[,"condition"] = conditions
pca_uncorrected[,"library_method"] = library_methods
pca_uncorrected[,"replicate"] = replicates

#plot the PCA
#create a classic 2-dimension PCA plot (first two principal components) with conditions and library methods indicated
cols <- c("UHR" = "#481567FF", "HBR" = "#1F968BFF")
p1 = ggplot(data=pca_uncorrected, aes(x=PC1, y=PC2, color=condition, shape=library_method))
p1 = p1 + geom_point(size=3)
p1 = p1 + stat_ellipse(type="norm", linetype=2)
p1 = p1 + labs(title="PCA, RNA-seq counts for 16 HBR/UHR and Ribo/PolyA samples (uncorrected data)", color="Condition", shape="Library Method")
p1 = p1 + scale_colour_manual(values = cols)

,Gene,Chr,UHR_Ribo_1,UHR_Ribo_2,UHR_Ribo_3,UHR_Ribo_4,HBR_Ribo_1,HBR_Ribo_2,HBR_Ribo_3,HBR_Ribo_4,UHR_Poly_1,UHR_Poly_2,UHR_Poly_3,UHR_Poly_4,HBR_Poly_1,HBR_Poly_2,HBR_Poly_3,HBR_Poly_4
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,A1BG,chr19,1,6,5,2,19,27,11,14,125,41,17,28,47,32,68,15
2,A1CF,chr10,527,357,310,446,7,12,4,3,543,696,549,736,2,0,2,1
3,A2M,chr12,19596,13416,9255,9993,8658,10623,6293,5661,44912,51246,40613,54269,13173,13426,17918,12806
4,A4GALT,chr22,8,2,6,7,14,10,12,0,140,78,40,47,43,55,97,58
5,A4GNT,chr3,0,10,8,5,6,6,6,1,6,8,4,0,9,2,12,4
6,AAAS,chr12,628,481,281,311,522,648,373,321,3186,2514,1580,2303,1528,1870,2093,1417


[1] 16104    18

# Bioconductor SVA and ComBat-Seq

ComBat-Seq is part of the [Surrogate Variable Analysis](https://www.bioconductor.org/packages/release/bioc/html/sva.html) package.

You can read more about the basic usage of ComBat-Seq on the [website](https://rnabio.org/module-03-expression/0003/05/01/Batch-Correction/).

Run the R code in the cell below. Uncomment all lines with ctrl + / if needed.

In [12]:
#perform the batch correction

#first we need to transform the format of our groups and batches from names (e.g. "UHR", "HBR", etc.) to numbers (e.g. 1, 2, etc.)
#in the command below "sapply" is used to apply the "switch" command to each element and convert names to numbers as we define
groups = sapply(as.character(conditions), switch, "UHR" = 1, "HBR" = 2, USE.NAMES = F)
batches = sapply(as.character(library_methods), switch, "Ribo" = 1, "Poly" = 2, USE.NAMES = F)

#now run ComBat_seq
corrected_data = ComBat_seq(counts = as.matrix(uncorrected_data[,sample_names]), batch = batches, group = groups)

#join the gene and chromosome names onto the now corrected counts from ComBat_seq
corrected_data = cbind(uncorrected_data[,c("Gene","Chr")], corrected_data)

#compare dimensions of corrected and uncorrected data sets
dim(uncorrected_data)
dim(corrected_data)

#visually compare values of corrected and uncorrected data sets
head(uncorrected_data)
head(corrected_data)

Found 2 batches
Using full model in ComBat-seq.
Adjusting for 1 covariate(s) or covariate level(s)
Estimating dispersions
Fitting the GLM model
Shrinkage off - using GLM estimates for parameters
Adjusting the data


[1] 16104    18

[1] 16104    18

,Gene,Chr,UHR_Ribo_1,UHR_Ribo_2,UHR_Ribo_3,UHR_Ribo_4,HBR_Ribo_1,HBR_Ribo_2,HBR_Ribo_3,HBR_Ribo_4,UHR_Poly_1,UHR_Poly_2,UHR_Poly_3,UHR_Poly_4,HBR_Poly_1,HBR_Poly_2,HBR_Poly_3,HBR_Poly_4
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,A1BG,chr19,1,6,5,2,19,27,11,14,125,41,17,28,47,32,68,15
2,A1CF,chr10,527,357,310,446,7,12,4,3,543,696,549,736,2,0,2,1
3,A2M,chr12,19596,13416,9255,9993,8658,10623,6293,5661,44912,51246,40613,54269,13173,13426,17918,12806
4,A4GALT,chr22,8,2,6,7,14,10,12,0,140,78,40,47,43,55,97,58
5,A4GNT,chr3,0,10,8,5,6,6,6,1,6,8,4,0,9,2,12,4
6,AAAS,chr12,628,481,281,311,522,648,373,321,3186,2514,1580,2303,1528,1870,2093,1417


,Gene,Chr,UHR_Ribo_1,UHR_Ribo_2,UHR_Ribo_3,UHR_Ribo_4,HBR_Ribo_1,HBR_Ribo_2,HBR_Ribo_3,HBR_Ribo_4,UHR_Poly_1,UHR_Poly_2,UHR_Poly_3,UHR_Poly_4,HBR_Poly_1,HBR_Poly_2,HBR_Poly_3,HBR_Poly_4
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,A1BG,chr19,1,11,9,3,33,60,15,27,52,23,12,18,36,27,49,17
2,A1CF,chr10,445,299,261,379,7,11,4,3,642,809,660,869,3,0,3,1
3,A2M,chr12,22354,15357,11183,12873,9013,11053,6540,5923,45040,50205,35144,50255,11144,11984,16118,10820
4,A4GALT,chr22,18,6,13,16,28,22,24,0,75,36,16,19,18,25,46,26
5,A4GNT,chr3,0,8,6,4,5,5,5,1,9,12,6,0,13,3,18,6
6,AAAS,chr12,953,743,403,424,794,989,567,481,2000,1652,1094,1557,1052,1246,1424,982


Perform PCA analysis and compare the batch corrected data with uncorrected

In [13]:
#calculate principal components for the uncorrected data
pca_corrected_obj = prcomp(corrected_data[,sample_names])

#pull PCA values out of the PCA object
pca_corrected = as.data.frame(pca_corrected_obj[2]$rotation)

#assign labels to the data frame
pca_corrected[,"condition"] = conditions
pca_corrected[,"library_method"] = library_methods
pca_corrected[,"replicate"] = replicates

#as above, create a PCA plot for comparison to the uncorrected data
cols <- c("UHR" = "#481567FF", "HBR" = "#1F968BFF")
p2 = ggplot(data=pca_corrected, aes(x=PC1, y=PC2, color=condition, shape=library_method))
p2 = p2 + geom_point(size=3)
p2 = p2 + stat_ellipse(type="norm", linetype=2)
p2 = p2 + labs(title="PCA, RNA-seq counts for 16 HBR/UHR and Ribo/PolyA samples (batch corrected data)", color="Condition", shape="Library Method")
p2 = p2 + scale_colour_manual(values = cols)

pdf(file="/home/user/bioinformatics/batch_correction/Uncorrected-vs-BatchCorrected-PCA.pdf")
grid.arrange(p1, p2, nrow = 2)
dev.off()

png 
  2

### DE analysis of corrected vs uncorrected

Run the R code below.

In [14]:
#perform differential expression analysis on the uncorrected data and batch corrected data sets

#first define the sets of samples to be compared to each other
uhr_ribo_samples = c("UHR_Ribo_1", "UHR_Ribo_2", "UHR_Ribo_3", "UHR_Ribo_4")
uhr_poly_samples = c("UHR_Poly_1", "UHR_Poly_2", "UHR_Poly_3", "UHR_Poly_4")
hbr_ribo_samples = c("HBR_Ribo_1", "HBR_Ribo_2", "HBR_Ribo_3", "HBR_Ribo_4")
hbr_poly_samples = c("HBR_Poly_1", "HBR_Poly_2", "HBR_Poly_3", "HBR_Poly_4")
uhr_samples = c(uhr_ribo_samples, uhr_poly_samples)
hbr_samples = c(hbr_ribo_samples, hbr_poly_samples)

#create a function that will run edgeR (DE analysis) for a particular pair of sample sets
run_edgeR = function(data, group_a_name, group_a_samples, group_b_samples, group_b_name){
  #create a list of all samples for this current comparison
  samples_for_comparison = c(group_a_samples, group_b_samples)
  
  #define the class factor for this pair of sample sets
  class = factor(c(rep(group_a_name,length(group_a_samples)), rep(group_b_name,length(group_b_samples))))
  
  #create a simplified data matrix for only these samples
  rawdata = data[,samples_for_comparison]
  
  #store gene names for later
  genes = rownames(data)
  gene_names = data[,"Gene"]
  
  #make DGElist object
  y = DGEList(counts=rawdata, genes=genes, group=class)
  
  #perform TMM normalization
  y <- calcNormFactors(y)
  
  #estimate dispersion
  y <- estimateCommonDisp(y, verbose=TRUE)
  y <- estimateTagwiseDisp(y)
  
  #differential expression test
  et <- exactTest(y)
  
  #print number of up/down significant genes at FDR = 0.05 significance level and store the DE status in a new variable (de)
  summary(de <- decideTestsDGE(et, p=.05))
  summary(de <- decideTestsDGE(et, adjust.method="fdr", p=.05))
  
  #create a matrix of significant DE genes
  mat <- cbind(
    genes, gene_names,
    sprintf('%0.3f', log10(et$table$PValue)),
    sprintf('%0.3f', et$table$logFC)
  )[as.logical(de),]
  colnames(mat) <- c("Gene", "Gene_Name", "Log10_Pvalue", "Log_fold_change")
  
  #order by log fold change
  o <- order(et$table$logFC[as.logical(de)],decreasing=TRUE)
  mat <- mat[o,]

  #fix the issue where corrected p-values that are 0 become -Inf upon log10 conversion
  x = as.numeric(mat[,"Log10_Pvalue"])
  lowest_pvalue = min(x[which(!x == -Inf)])
  i = which(mat[,"Log10_Pvalue"] == -Inf)
  mat[i,"Log10_Pvalue"] = lowest_pvalue
  
  return(mat)
}

#run the five comparisons through edgeR using the *uncorrected data*
uhr_ribo_vs_hbr_ribo_uncorrected = run_edgeR(data=uncorrected_data, group_a_name="UHR", group_a_samples=uhr_ribo_samples, group_b_name="HBR", group_b_samples=hbr_ribo_samples)
uhr_poly_vs_hbr_poly_uncorrected = run_edgeR(data=uncorrected_data, group_a_name="UHR", group_a_samples=uhr_poly_samples, group_b_name="HBR", group_b_samples=hbr_poly_samples)
uhr_ribo_vs_hbr_poly_uncorrected = run_edgeR(data=uncorrected_data, group_a_name="UHR", group_a_samples=uhr_ribo_samples, group_b_name="HBR", group_b_samples=hbr_poly_samples)
uhr_poly_vs_hbr_ribo_uncorrected = run_edgeR(data=uncorrected_data, group_a_name="UHR", group_a_samples=uhr_poly_samples, group_b_name="HBR", group_b_samples=hbr_ribo_samples)
uhr_vs_hbr_uncorrected = run_edgeR(data=uncorrected_data, group_a_name="UHR", group_a_samples=uhr_samples, group_b_name="HBR", group_b_samples=hbr_samples)

#run the same five comparisons through edgeR using the *batch corrected data*
uhr_ribo_vs_hbr_ribo_corrected = run_edgeR(data=corrected_data, group_a_name="UHR", group_a_samples=uhr_ribo_samples, group_b_name="HBR", group_b_samples=hbr_ribo_samples)
uhr_poly_vs_hbr_poly_corrected = run_edgeR(data=corrected_data, group_a_name="UHR", group_a_samples=uhr_poly_samples, group_b_name="HBR", group_b_samples=hbr_poly_samples)
uhr_ribo_vs_hbr_poly_corrected = run_edgeR(data=corrected_data, group_a_name="UHR", group_a_samples=uhr_ribo_samples, group_b_name="HBR", group_b_samples=hbr_poly_samples)
uhr_poly_vs_hbr_ribo_corrected = run_edgeR(data=corrected_data, group_a_name="UHR", group_a_samples=uhr_poly_samples, group_b_name="HBR", group_b_samples=hbr_ribo_samples)
uhr_vs_hbr_corrected = run_edgeR(data=corrected_data, group_a_name="UHR", group_a_samples=uhr_samples, group_b_name="HBR", group_b_samples=hbr_samples)

#how much of a difference does batch correction make when doing the comparison of all UHR vs all HBR samples?
dim(uhr_vs_hbr_uncorrected)
dim(uhr_vs_hbr_corrected)

#create upset plots to summarize the overlap between the comparisons performed above

#first create upset plot from the uncorrected data
pdf(file="/home/user/bioinformatics/batch_correction/Uncorrected-UpSet.pdf")
listInput = list("4 UHR Ribo vs 4 HBR Ribo" = uhr_ribo_vs_hbr_ribo_uncorrected[,"Gene"], 
                 "4 UHR Poly vs 4HBR Poly" = uhr_poly_vs_hbr_poly_uncorrected[,"Gene"],
                 "4 UHR Ribo vs 4 HBR Poly" = uhr_ribo_vs_hbr_poly_uncorrected[,"Gene"],
                 "4 UHR Poly vs 4 HBR Ribo" = uhr_poly_vs_hbr_ribo_uncorrected[,"Gene"],
                 "8 UHR vs 8 HBR" = uhr_vs_hbr_uncorrected[,"Gene"])
upset(fromList(listInput), order.by = "freq", number.angles=45, point.size=3)
dev.off()

#now create an upset plot from the batch corrected data
pdf(file="/home/user/bioinformatics/batch_correction/BatchCorrected-UpSet.pdf")
listInput = list("4 UHR Ribo vs 4 HBR Ribo" = uhr_ribo_vs_hbr_ribo_corrected[,"Gene"], 
                 "4 UHR Poly vs 4 HBR Poly" = uhr_poly_vs_hbr_poly_corrected[,"Gene"],
                 "4 UHR Ribo vs 4 HBR Poly" = uhr_ribo_vs_hbr_poly_corrected[,"Gene"],
                 "4 UHR Poly vs 4 HBR Ribo" = uhr_poly_vs_hbr_ribo_corrected[,"Gene"],
                 "8 UHR vs 8 HBR" = uhr_vs_hbr_corrected[,"Gene"])
upset(fromList(listInput), order.by = "freq", number.angles=45, point.size=3)
dev.off()

#write out the final set of DE genes where all UHR and HBR samples were compared using the corrected data
write.table(uhr_vs_hbr_corrected, file="/home/user/bioinformatics/batch_correction/DE_genes_uhr_vs_hbr_corrected.tsv", quote=FALSE, row.names=FALSE, sep="\t")

Disp = 0.014 , BCV = 0.1183 
Disp = 0.04558 , BCV = 0.2135 
Disp = 0.02124 , BCV = 0.1457 
Disp = 0.03868 , BCV = 0.1967 
Disp = 0.20008 , BCV = 0.4473 
Disp = 0.02736 , BCV = 0.1654 
Disp = 0.03426 , BCV = 0.1851 
Disp = 0.02728 , BCV = 0.1652 
Disp = 0.03388 , BCV = 0.1841 
Disp = 0.03917 , BCV = 0.1979 


[1] 10200     4

[1] 12489     4

png 
  2

png 
  2

### Important considerations when interpreting uncorrected vs corrected

Differenes in library construction can make it appear that there are more DE genes (or maybe less) than how many DE genes there actually are. This is due to differences in library construction, not differences in biological variability.

After batch correction, the different preparation methods should agree better on what genes are DE.

Combining all 8 of the batch corrected samples gives a higher statistical power and shows more DE genes than the 4 v 4 comparison. This is probably the most accurate comparison. 

***
# DE Pathway Analysis

The gene-set enrichment and pathway analysis will be done using [GAGE](https://bioconductor.org/packages/release/bioc/html/gage.html), the Generally Applicable Gene-set Enrichment tool, a bioconductor package. 

Start by changing directories to the raw count files generated by htseq-count previously.

```cd /home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts```

Run the R code below to import libs and set working directory.

In [16]:
library(AnnotationDbi)
library(org.Hs.eg.db)
library(GO.db)
library(gage)

setwd ("/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/")

### Setting up gene set databases

We need a reference for what gene sets our DE genes may actually be a part of. There are many sources for such databases such as [GO](http://www.geneontology.org/), [KEGG](https://www.kegg.jp/), [MSigDB](https://www.gsea-msigdb.org/gsea/msigdb), and [WikiPathways](https://www.wikipathways.org/index.php/WikiPathways).

In [22]:
# Set up go database
go.hs <- go.gsets(species="human")
go.bp.gs <- go.hs$go.sets[go.hs$go.subs$BP]
go.mf.gs <- go.hs$go.sets[go.hs$go.subs$MF]
go.cc.gs <- go.hs$go.sets[go.hs$go.subs$CC]

# Here we will read in an MSigDB gene set that was selected for this exercise and saved to the course website. 
c8 <-"http://genomedata.org/rnaseq-tutorial/c8.all.v7.2.entrez.gmt"
all_cell_types <-readList(c8)

Gene ID type for 'human' is: 'EG'



In [23]:
# Read in DE results from edgeR
DE_genes <-read.table("/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/DE_genes.txt",sep="\t",header=T,stringsAsFactors = F)

### Annotating Genes

The Ensembl IDs in our DE gene list need to be converted to Entrez gene IDs to continue with the pathway analysis.

Bioconductor maintains annotation data for many species in the [OrgDb](https://bioconductor.org/packages/release/BiocViews.html#___OrgDb). Just use the ```mapIds()``` function in R. ```multiVals="first"``` means only the first identifier should be returned if there are multiple.

In [24]:
DE_genes$entrez <- mapIds(org.Hs.eg.db, keys=DE_genes$Gene, column="ENTREZID", keytype="ENSEMBL", multiVals="first")

'select()' returned 1:many mapping between keys and columns



### Identifier mapping

There may be some Ensembl IDs that were not mapped to Entrez IDs. This is largely due to differences in how different sources annotate their genes. The next section removes the ERCC spike-in genes and creates a different identifier for further mapping.

In [25]:
#Remove spike-in
DE_genes_clean <- DE_genes[!grepl("ERCC",DE_genes$Gene),]

##Just so we know what we have removed 
ERCC_gene_count <-nrow(DE_genes[grepl("ERCC",DE_genes$Gene),])
ERCC_gene_count

###Deal with genes that we do not have an Entrez ID for 
missing_ensembl_key<-DE_genes_clean[is.na(DE_genes_clean$entrez),]
DE_genes_clean <-DE_genes_clean[!DE_genes_clean$Gene %in% missing_ensembl_key$Gene,]

###Try mapping using a different key
missing_ensembl_key$entrez <- mapIds(org.Hs.eg.db, keys=missing_ensembl_key$Symbol, column="ENTREZID", keytype="SYMBOL", multiVal='first')

#Remove remaining genes 
missing_ensembl_key_update <- missing_ensembl_key[!is.na(missing_ensembl_key$entrez),]

#Create a Final Gene list of all genes where we were able to find an Entrez ID (using two approaches)
DE_genes_clean <-rbind(DE_genes_clean,missing_ensembl_key_update)

[1] 32

'select()' returned 1:1 mapping between keys and columns



### Final prep of edgeR results for gage

Assign log2 fold change values to the Entrez gene IDs.

In [26]:
# grab the log fold changes for everything
De_gene.fc <- DE_genes_clean$logFC

# set the name for each row to be the Entrez Gene ID
names(De_gene.fc) <- DE_genes_clean$entrez

### Running pathway analysis

The moment we've all been waiting for. Now we can use gage() to see what pathways are actually affected.

Note on the abbreviations below: “bp” refers to biological process, “mf” refers to molecular function, and “cc” refers to cellular process. These are the three main categories of gene ontology terms/annotations that were mentioned above.

In [27]:
#Run GAGE
#go 
fc.go.bp.p <- gage(De_gene.fc, gsets = go.bp.gs)
fc.go.mf.p <- gage(De_gene.fc, gsets = go.mf.gs)
fc.go.cc.p <- gage(De_gene.fc, gsets = go.cc.gs)

#msigdb
fc.go.c8.p <- gage(De_gene.fc, gsets =all_cell_types)

###Convert to dataframes 
fc.go.bp.p.up <- as.data.frame(fc.go.bp.p$greater)
fc.go.mf.p.up <- as.data.frame(fc.go.mf.p$greater)
fc.go.cc.p.up <- as.data.frame(fc.go.cc.p$greater)

fc.go.bp.p.down <- as.data.frame(fc.go.bp.p$less)
fc.go.mf.p.down <- as.data.frame(fc.go.mf.p$less)
fc.go.cc.p.down <- as.data.frame(fc.go.cc.p$less)

fc.go.c8.p.up <- as.data.frame(fc.go.c8.p$greater)
fc.go.c8.p.down <- as.data.frame(fc.go.c8.p$less)

### Explore significant results

In [28]:
#At this point we will give you a hint. Look at the cellular process results. Try doing something like this to find some significant results: 

head(fc.go.cc.p.down[order(fc.go.cc.p.down$p.val),], n=20)

#Do this with more of your results files to see what else you have uncovered

#You can do the same thing with your results from MSigDB

head(fc.go.c8.p.up)
head(fc.go.c8.p.down)

,p.geomean,stat.mean,p.val,q.val,set.size,exp1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GO:0043005 neuron projection,0.002830810,-2.9261518,0.002830810,0.1763434,21,0.002830810
GO:0045202 synapse,0.003962774,-2.7855257,0.003962774,0.1763434,23,0.003962774
GO:0030054 cell junction,0.006286971,-2.5605455,0.006286971,0.1865135,37,0.006286971
GO:0036477 somatodendritic compartment,0.009410291,-2.5275692,0.009410291,0.2093790,13,0.009410291
GO:0031226 intrinsic component of plasma membrane,0.018494522,-2.2088116,0.018494522,0.3292025,13,0.018494522
GO:0005887 integral component of plasma membrane,0.025142041,-2.0716391,0.025142041,0.3309661,12,0.025142041
GO:0098590 plasma membrane region,0.026031041,-2.0056064,0.026031041,0.3309661,20,0.026031041
GO:0042995 cell projection,0.045214067,-1.7149640,0.045214067,0.4599591,39,0.045214067
GO:0120025 plasma membrane bounded cell projection,0.049487969,-1.6708696,0.049487969,0.4599591,38,0.049487969


,p.geomean,stat.mean,p.val,q.val,set.size,exp1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MURARO_PANCREAS_ACINAR_CELL,0.03055952,2.0308447,0.03055952,0.5149578,11,0.03055952
MANNO_MIDBRAIN_NEUROTYPES_HMGL,0.08514341,1.4413268,0.08514341,0.5149578,10,0.08514341
HAY_BONE_MARROW_ERYTHROBLAST,0.10023004,1.2943299,0.10023004,0.5149578,32,0.10023004
MANNO_MIDBRAIN_NEUROTYPES_HPERIC,0.10299725,1.3146493,0.10299725,0.5149578,11,0.10299725
MURARO_PANCREAS_DUCTAL_CELL,0.11703587,1.2219728,0.11703587,0.5149578,15,0.11703587
HAY_BONE_MARROW_NK_CELLS,0.36679715,0.3457741,0.36679715,0.9999306,10,0.36679715


,p.geomean,stat.mean,p.val,q.val,set.size,exp1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MANNO_MIDBRAIN_NEUROTYPES_HGABA,6.938946e-05,-4.364784,6.938946e-05,0.001526568,18,6.938946e-05
MANNO_MIDBRAIN_NEUROTYPES_HSERT,5.349125e-03,-2.961767,5.349125e-03,0.029188378,10,5.349125e-03
ZHONG_PFC_C3_MICROGLIA,6.979205e-03,-2.727035,6.979205e-03,0.029188378,10,6.979205e-03
MANNO_MIDBRAIN_NEUROTYPES_HNBGABA,7.126965e-03,-2.670036,7.126965e-03,0.029188378,12,7.126965e-03
ZHONG_PFC_C3_ASTROCYTE,7.495148e-03,-2.615467,7.495148e-03,0.029188378,14,7.495148e-03
MANNO_MIDBRAIN_NEUROTYPES_HDA,7.960467e-03,-2.661157,7.960467e-03,0.029188378,10,7.960467e-03


### Export to .tsv

In [29]:
write.table(fc.go.cc.p.up,"/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/fc.go.cc.p.up.tsv",quote = F,sep = "\t",col.names = T,row.names = T)
write.table(fc.go.cc.p.down,"/home/user/bioinformatics/ref_genome/test_data/align_output/de/htseq_counts/fc.go.cc.p.down.tsv",quote = F,sep = "\t",col.names = T,row.names = T)

### Visualize

This will be done using [GOView](http://www.webgestalt.org/2017/GOView/), part of [WEB-based Gene Set Analysis ToolKit (WebGestalt)](http://www.webgestalt.org/).

Simply go to the GOView link above and use the data from the ```fc.go.cc.p.up.tsv``` and ```fc.go.cc.p.down.tsv``` files. Follow the instructions on the page to visualize. 

# The End

This brings the pipeline to an end, but more will surely be added and tweaked in the future.